In [1]:
# import package 
import time 
import numpy as np
import pandas as pd

from scipy.stats import norm

In [2]:
def black_sholes(inputs,type_option):
    """
    European option according to the Black-Scholes model
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    r = inputs['r']
    start = time.perf_counter()
    
    d1 = (np.log(x0/k) + (r + 0.5 * sigma**2) * T) / sigma / np.sqrt(T)
    d2 = d1 - sigma * np.sqrt(T)
    
    if type_option == "call":
        price = x0 * norm.cdf(d1) - k * np.exp(-r*T) * norm.cdf(d2)
    else:
        price = k * exp(-r*T) * norm.cdf(-d2) - x0 * norm.cdf(-d1)
        
    end = time.perf_counter() - start
    
    return price,np.nan,end

In [3]:
def black_sholes_geometric_asian(inputs,type_option):
    """
    Asian option with geometric mean 
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    r = inputs['r']
    y = 0
    
    y = 0.5 * (r + y + sigma**2 / 6)
    sigma = sigma / np.sqrt(3)
    
    d1 = (np.log(x0/k) + (r + 0.5 * sigma**2) * T) / sigma / np.sqrt(T)
    d2 = d1 - sigma * np.sqrt(T)
    
    if type_option == "call":
        price = x0 * np.exp(-y * T) * norm.cdf(d1) - k * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = k * exp(-r * T) * norm.cdf(-d2) - x0 * norm.cdf(-d1) * np.exp(-y * T)
        
    return price

In [4]:
def simulation_monte_carlo_asian_option(inputs,type_option):
    """
    Asian option with arithmetic mean 
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    d = inputs['steps']
    r = inputs['r']
    
    N = inputs['N'] # number of paths to generate 
    alpha = inputs['Confidence level']
    seed = inputs['seed']
    
    dt = T / d
    
    if not np.isnan(seed):
        np.random.seed((int(seed)))
    
    start = time.perf_counter()
    
    z = np.random.normal(size=(N,d))
    
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt) * z
    
    # Simulation of the underlying asset price
    delta_X = np.exp(drift + diffusion)
    X = np.concatenate((np.ones((N,1)) * x0, delta_X),axis=1)
    X = np.cumprod(X,axis=1)

    # option pricing 
    X_average = np.mean(X,axis=1)
    
    factor_discount = np.exp(-r * T)
    if type_option == "call":
        list_price = factor_discount * np.maximum(X_average - k, 0)
    else:
        list_price = factor_discount * np.maximum(k - X_average, 0)
    
    price = np.mean(list_price)
    
    # unbiased standard deviation, degree of freedom N-1
    std_price = np.std(list_price, ddof=1)
    # margin error 
    delta = norm.ppf(1-(1-alpha)/2) * std_price / np.sqrt(N)
    
    end = time.perf_counter() - start
    
    return price, delta, end

In [5]:
def simulation_monte_carlo_asian_option_cv(inputs,type_option):
    """
    Asian option with arithmetic mean
    Geometirc-mean Asian option ==> control variate  
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    d = inputs['steps']
    r = inputs['r']
    
    N = inputs['N'] # number of paths to generate 
    n = inputs['n']
    alpha = inputs['Confidence level']
    seed = inputs['seed']
    
    dt = T / d
    
    if not np.isnan(seed):
        np.random.seed((int(seed)))
    
    start = time.perf_counter()
    
    z = np.random.normal(size=(N,d))
    
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt) * z
    
    # Simulation of the underlying asset price
    delta_X = np.exp(drift + diffusion)
    X = np.concatenate((np.ones((N,1)) * x0, delta_X),axis=1)
    X = np.cumprod(X,axis=1)

    # option pricing 
    X_average = np.mean(X,axis=1)
    X_g_average = np.exp(np.mean(np.log(X),axis=1))
    
    factor_discount = np.exp(-r * T)
    if type_option == "call":
        list_price = factor_discount * np.maximum(X_average - k, 0)
        list_price_g = factor_discount * np.maximum(X_g_average - k, 0)        
    else:
        list_price = factor_discount * np.maximum(k - X_average, 0)
        list_price_g = factor_discount * np.maximum(k - X_g_average, 0)
    
    price = np.mean(list_price)

    # estimator of control variate technique 
    price_asian_g = black_sholes_geometric_asian(inputs,type_option)
    C = list_price_g - price_asian_g
    
    # regression Y on C by n-size sample 
    beta_estimated = np.cov(list_price[:n],C[:n])[0,1] / np.var(C[:n])
    
    list_price = list_price[n:] - beta_estimated * C[n:]
    price = np.mean(list_price)
    
    # unbiased standard deviation, degree of freedom N-1
    std_price = np.std(list_price, ddof=1)
    # margin error 
    delta = norm.ppf(1-(1-alpha)/2) * std_price / np.sqrt(N)
    
    end = time.perf_counter() - start
    
    return price, delta, end

In [27]:
def simulation_monte_carlo_asian_option_av(inputs,type_option):
    """
    Asian option with arithmetic mean
    Control variate + Antithetique variate technique 
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    d = inputs['steps']
    r = inputs['r']
    
    N = inputs['N'] # number of paths to generate 
    alpha = inputs['Confidence level']
    seed = inputs['seed']
    
    dt = T / d
    
    if not np.isnan(seed):
        np.random.seed((int(seed)))
    
    start = time.perf_counter()
    
    z = np.random.normal(size=(N,d))
    
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt) * z
    diffusion_a = sigma * np.sqrt(dt) * (-z)
    
    # Simulation of the underlying asset price
    delta_X = np.exp(drift + diffusion)
    X = np.concatenate((np.ones((N,1)) * x0, delta_X),axis=1)
    X = np.cumprod(X,axis=1)

    delta_X_a = np.exp(drift + diffusion_a)
    X_a = np.concatenate((np.ones((N,1)) * x0, delta_X_a),axis=1)
    X_a = np.cumprod(X_a,axis=1)
    
    # option pricing 
    X_average = np.mean(X,axis=1)
    X_a_average = np.mean(X_a,axis=1)
    
    factor_discount = np.exp(-r * T)
    if type_option == "call":
        list_price = factor_discount * np.maximum(X_average - k, 0)
        list_price_a = factor_discount * np.maximum(X_a_average - k, 0)

    else:
        list_price = factor_discount * np.maximum(k - X_average, 0)
        list_price_a = factor_discount * np.maximum(X_a_average - k, 0)
    
    list_price = 0.5 * (list_price + list_price_a)
    price = np.mean(list_price)
    
    # unbiased standard deviation, degree of freedom N-1
    std_price = np.std(list_price, ddof=1)
    # margin error 
    delta = norm.ppf(1-(1-alpha)/2) * std_price / np.sqrt(N)
    
    end = time.perf_counter() - start
    
    return price, delta, end

In [28]:
def simulation_monte_carlo_asian_option_cv_av(inputs,type_option):
    """
    Asian option with arithmetic mean
    Antithetique variate
    """
    x0 = inputs['x0']
    k = inputs['K']
    sigma = inputs['sigma']
    T = inputs['T']
    d = inputs['steps']
    r = inputs['r']
    
    N = inputs['N'] # number of paths to generate
    n = inputs['n']
    alpha = inputs['Confidence level']
    seed = inputs['seed']
    
    dt = T / d
    
    if not np.isnan(seed):
        np.random.seed((int(seed)))
    
    start = time.perf_counter()
    
    z = np.random.normal(size=(N,d))
    
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt) * z
    diffusion_a = sigma * np.sqrt(dt) * (-z)
    
    # Simulation of the underlying asset price
    delta_X = np.exp(drift + diffusion)
    X = np.concatenate((np.ones((N,1)) * x0, delta_X),axis=1)
    X = np.cumprod(X,axis=1)

    delta_X_a = np.exp(drift + diffusion_a)
    X_a = np.concatenate((np.ones((N,1)) * x0, delta_X_a),axis=1)
    X_a = np.cumprod(X_a,axis=1)
    
    # option pricing 
    X_average = np.mean(X,axis=1)
    X_a_average = np.mean(X_a,axis=1)

    X_g_average = np.exp(np.mean(np.log(X),axis=1))
    X_g_a_average = np.exp(np.mean(np.log(X_a),axis=1))
    
    factor_discount = np.exp(-r * T)
    if type_option == "call":
        list_price = factor_discount * np.maximum(X_average - k, 0)
        list_price_a = factor_discount * np.maximum(X_a_average - k, 0)        
        list_price_g = factor_discount * np.maximum(X_g_average - k, 0)
        list_price_g_a = factor_discount * np.maximum(X_g_a_average - k, 0)

    else:
        list_price = factor_discount * np.maximum(k - X_average, 0)
        list_price_a = factor_discount * np.maximum(k - X_a_average, 0)        
        list_price_g = factor_discount * np.maximum(k - X_g_average, 0)
        list_price_g_a = factor_discount * np.maximum(k - X_g_a_average, 0)
   

    # estimator of control variate technique 
    price_asian_g = black_sholes_geometric_asian(inputs,type_option)
    
    Y_a = 0.5 * (list_price + list_price_a) 
    C_a = list_price_g - 2 * price_asian_g + list_price_g_a
    
    # regression Y on C by n-size sample 
    beta_estimated = np.cov(Y_a[:n],C_a[:n])[0,1] / np.var(C_a[:n])
    
    list_price = Y_a[n:] - beta_estimated * C_a[n:]
    price = np.mean(list_price)
        
    # unbiased standard deviation, degree of freedom N-1
    std_price = np.std(list_price, ddof=1)
    # margin error 
    delta = norm.ppf(1-(1-alpha)/2) * std_price / np.sqrt(N)
    
    end = time.perf_counter() - start
    
    return price, delta, end

In [24]:
# inputs
inputs = {}
inputs['x0'] = 100
inputs['K'] = 110
inputs['sigma'] = 0.3
inputs['T'] = 1
inputs['r'] = 0.05

inputs['N'] = 50000
inputs['steps'] = 100
inputs['Confidence level'] = 0.95
inputs['seed'] = 1

inputs['n'] = 20000

In [7]:
inputs

{'x0': 100,
 'K': 110,
 'sigma': 0.3,
 'T': 1,
 'r': 0.05,
 'N': 50000,
 'steps': 100,
 'Confidence level': 0.95,
 'seed': 1,
 'n': 20000}

In [29]:
results = {}

results["Call Monte Carlo"] = simulation_monte_carlo_asian_option(inputs,"call")
results["Call Monte Carlo CV"] = simulation_monte_carlo_asian_option_cv(inputs,"call")
results["Call Monte Carlo AV"] = simulation_monte_carlo_asian_option_av(inputs,"call")
results["Call Monte Carlo CV+AV"] = simulation_monte_carlo_asian_option_cv_av(inputs,"call")



In [30]:
pd.options.display.float_format = '{:,.8f}'.format
pd.DataFrame(results,index=["value","error","time"])


,Call Monte Carlo,Call Monte Carlo CV,Call Monte Carlo AV,Call Monte Carlo CV+AV
value,4.06521338,3.94808682,4.05475457,3.95242053
error,0.07794536,0.00395951,0.04908793,0.00281714
time,0.25673596,0.30295221,0.36710113,0.47155095
